In [2]:
from dexperts import DExperts

In [20]:
dexperts = DExperts(
    base_model='gpt2-medium',
    antiexpert_model='eliolio/gpt2-finetuned-redditbias',
    expert_model='eliolio/gpt2-finetuned-reddit-antibias',
    tokenizer='gpt2',
)

dexperts_void = DExperts(
    base_model='gpt2-medium',
    tokenizer='gpt2',
)

In [8]:
def generate_text(prompt, typical_p=None, top_p=None):
    inputs = dexperts.tokenizer(prompt, return_tensors='pt')
    out = dexperts.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        num_return_sequences=3,
        max_new_tokens=15,
        do_sample=True,
        top_p=top_p,
        typical_p=typical_p,
    )
    return [dexperts.tokenizer.decode(out[i], skip_special_tokens=True) for i in range(len(out))]

In [9]:
generate_text(prompt='I am very')

['I am very confident that any attempt to pass legislation supporting recreational marijuana on the Canadian public will',
 'I am very virtuous," he responded. "So virtuous that if I save 20 persons,',
 'I am very confident."\n\nLiam Tipton, executive vice president of the']

In [22]:
dexperts("I am very", alpha=0.0)

{'logits': tensor([[[ -78.3232,  -77.6039,  -80.8931,  ...,  -89.6521,  -82.6672,
            -80.4394],
          [-100.7663, -100.0332, -105.9358,  ..., -109.0896, -108.7145,
           -102.2539],
          [-118.6408, -118.0217, -124.4341,  ..., -125.5256, -125.2729,
           -120.2754]]]),
 'perplexity': tensor(1828.4690)}

In [23]:
dexperts_void("I am very", alpha=10.0)

{'logits': tensor([[[ -78.3232,  -77.6039,  -80.8931,  ...,  -89.6521,  -82.6672,
            -80.4394],
          [-100.7663, -100.0332, -105.9358,  ..., -109.0896, -108.7145,
           -102.2539],
          [-118.6408, -118.0217, -124.4341,  ..., -125.5256, -125.2729,
           -120.2754]]]),
 'perplexity': tensor(1828.4690)}